In [1]:
from pyspark.sql import SparkSession, functions
from pyspark.sql.types import *
import pandas as pd
import numpy as np

In [2]:
spark = SparkSession.builder.getOrCreate()

filename = "stream_60_2000.txt"

schema = StructType([\
    StructField("sid", IntegerType(), True),\
    StructField("ip", IntegerType(), True)])

text_file = spark.read.format("csv").option("header", "false").schema(schema).load(filename)
df_int = text_file.toPandas()

In [3]:
def create_dict(df_int):
    num_srv = 100
    srv_dict = {}
    for i_srv in range(0, num_srv):
        # list of all "ip" for "sid"==i_srv
        ip_list = df_int[df_int["sid"] == i_srv]["ip"].tolist()
        # sort IP in the list
        ip_list.sort()
        #print(ip_list)

        ip_dict = {}
        # init with invalid value
        ip_prev = -1
        # go through all IP addresses for server 'i_srv'
        for i_ip in range(0, len(ip_list)):
            # skipp counting for already counted IP
            if ip_list[i_ip] != ip_prev:
                #print(ip_list.count(ip_list[i_ip]))
                # add key 'ip' with value 'count of requests' to dictionary 'ip_dict'
                ip_dict[ip_list[i_ip]] = ip_list.count(ip_list[i_ip])
                # remember last checked IP
                ip_prev = ip_list[i_ip]

        # add new key 'i_srv' in dictionarry with value 'ip_dict'
        if ip_dict != {}:
            srv_dict[i_srv]=ip_dict
    #print(srv_dict)
    return srv_dict

In [4]:
def similarity_a_b(array_a, array_b):
    summation = 0
    array_a_keys = list(array_a.keys())
    array_b_keys = list(array_b.keys())
    j = 0
    k = 0
    for m in range(0,max(len(array_a_keys),len(array_b_keys))):
        if (array_a_keys[j] < array_b_keys[k]):
#            print("j:", j, ", k:", k, ", value ip" + str(array_a_keys[j]) + ":", array_a.get(array_a_keys[j]), ", value ip" + str(array_b_keys[k]) + ":", array_b.get(array_b_keys[k]))
            j = j + 1
        elif (array_a_keys[j] > array_b_keys[k]):
#            print("j:", j, ", k:", k, ", value ip" + str(array_a_keys[j]) + ":", array_a.get(array_a_keys[j]), ", value ip" + str(array_b_keys[k]) + ":", array_b.get(array_b_keys[k]))
            k = k + 1
        elif (array_a_keys[j] == array_b_keys[k]):
#            print("j:", j, ", k:", k, ", value ip" + str(array_a_keys[j]) + ":", array_a.get(array_a_keys[j]), ", value ip" + str(array_b_keys[k]) + ":", array_b.get(array_b_keys[k]))
            product = array_a.get(array_a_keys[j]) * array_b.get(array_b_keys[k])
            summation = summation + product
#            print("product:", product, ", summation:", summation)
            j = j + 1
            k = k + 1
        if (j >= len(array_a_keys) or k >= len(array_b_keys)):
            break
#    print("Final summation:", summation)
    if summation > 3000:
        return 1
    else:
        return 0           

In [5]:
def similarity_all(thisdict, num_serv):
    count = 0
    for i in range(0,num_serv):
        for j in range(i+1,num_serv):
            count = count + similarity_a_b(thisdict.get(i), thisdict.get(j))
    print(str(text_file.count()) + ",", count)

srv_dict = create_dict(df_int)
similarity_all(srv_dict, 100)

120000, 44
